# Normalization improvement

- here we demonstrate a utilization of `var_cache`, which helps with creating a more coherent result of normalization
- sometimes during normalization, due to the interplay of state sets and the determinization-like approach to the algorithm, some variable-labeled transitions can be created, which go against the variable order (i.e. create paths in the normalized UBDA, that allow for repeating variables or even going against the variable order)
- the proposed approach does not allow for such transitions to be created
- during the phase of normalization where a transition is being created, before it is actually added to the result, a check against the `var_cache` is performed
- `var_cache` contains last variable that a state set can see (state sets "survive" to next iterations when they are used to create some transition during last iteration)
- if the newly created transition would disrupt the variable order (i.e. `var_cache[source]` >= `var_cache[t]` where `t` is some target state of the transition), it is not created
- removing `old=True` option from normalization call uses the fix

In [ ]:
from tree_automata import TTreeAut
from tree_automata.var_manipulation import compress_vars
from helpers.string_manipulation import create_var_order_list

from formats.render_dot import convert_to_dot
from formats.format_vtf import import_treeaut_from_vtf
from canonization.normalization import ubda_normalize
from canonization.unfolding import ubda_unfolding

# Example 1

In [ ]:
from tree_automata.functions.trimming import remove_useless_states


example1: TTreeAut = import_treeaut_from_vtf("../tests/normalization/newNormTest4-loops.vtf")
example1.remove_transition('q1','q1-LH-[q1,q1]')
example1.remove_transition('q2','q2-LH-[q2,q2]')
convert_to_dot(example1)

In [ ]:
example1_unfolded: TTreeAut = ubda_unfolding(example1, 9)
example1_unfolded.reformat_states()
print(example1_unfolded)

convert_to_dot(example1_unfolded)

In [ ]:
# self loop q7 -> (q7, q12) allows for {q7} -x5-> ({q4,q7}, {q8,q12}) to be created
# self loop q6 -> (q10, q6) allows for {q6} -x3-> ({q5,q10}, {q3,q6}) to be created
# some variable checking during edge creation has to happen
# setting 'old=False' makes these transitions go away

example1_normalized: TTreeAut = ubda_normalize(example1_unfolded, create_var_order_list('x', 9), old=True)
convert_to_dot(compress_vars(example1_normalized))

In [ ]:
VARS = 4
example2: TTreeAut = import_treeaut_from_vtf("../tests/normalization/newNormTest4-loops.vtf")
example2.remove_transition('q1','q1-LH-[q1,q1]')
example2.remove_transition('q2','q2-LH-[q2,q2]')
example2.transitions['q2']['q2-LH-[q3,q4]'].info.variable = 'x2'
example2.transitions['q0']['q0-LH-[q1,q2]'].children[0] = 'q2'
example2.transitions['q0']['q0-LH-[q1,q2]'].info.box_array = [None, None]
example2 = remove_useless_states(example2)
example2.reformat_states()
example2.reformat_keys()
convert_to_dot(example2)

In [ ]:
example2_unfolded: TTreeAut = ubda_unfolding(example2, VARS)
example2_unfolded.reformat_states()
print(example2_unfolded)

convert_to_dot(example2_unfolded)

In [ ]:
example2_normalized: TTreeAut = ubda_normalize(example2_unfolded, create_var_order_list('x', VARS), old=True)
convert_to_dot(compress_vars(example2_normalized))